### WOA（Sample code）

In [ ]:
import numpy as np

class WhaleOptimizationAlgorithm:
    def __init__(self, obj_func, dim, lb, ub, max_iter=100, pop_size=30):
        self.obj_func = obj_func
        self.dim = dim
        self.lb = lb
        self.ub = ub
        self.max_iter = max_iter
        self.pop_size = pop_size
        
    def optimize(self):
        # 初始化参数
        a = 2.0  # 包围系数[1,2](@ref)
        b = 1.0  # 螺旋形状常数
        
        # 初始化种群
        positions = np.random.uniform(self.lb, self.ub, (self.pop_size, self.dim))
        fitness = np.array([self.obj_func(x) for x in positions])
        best_idx = np.argmin(fitness)
        best_pos = positions[best_idx].copy()
        best_fit = fitness[best_idx]
        convergence = []
        
        for t in range(self.max_iter):
            a_linear = 2 - t * (2 / self.max_iter)  # 线性递减参数[2](@ref)
            
            for i in range(self.pop_size):
                A = 2 * a_linear * np.random.rand() - a_linear
                C = 2 * np.random.rand()
                p = np.random.rand()
                
                # 包围猎物或螺旋更新[1](@ref)
                if p < 0.5:
                    if abs(A) < 1:
                        D = abs(C * best_pos - positions[i])
                        positions[i] = best_pos - A * D  # 收缩包围
                    else:
                        rand_idx = np.random.randint(0, self.pop_size)
                        X_rand = positions[rand_idx]
                        D = abs(C * X_rand - positions[i])
                        positions[i] = X_rand - A * D    # 全局搜索
                else:
                    distance = abs(best_pos - positions[i])
                    positions[i] = distance * np.exp(b * (2*np.pi*np.random.rand())) * np.cos(2*np.pi*np.random.rand()) + best_pos
                
                # 边界约束
                positions[i] = np.clip(positions[i], self.lb, self.ub)
                
                # 更新适应度
                new_fitness = self.obj_func(positions[i])
                if new_fitness < best_fit:
                    best_pos = positions[i].copy()
                    best_fit = new_fitness
            
            convergence.append(best_fit)
        
        return best_pos, best_fit, convergence

# 示例调用
woa = WhaleOptimizationAlgorithm(sphere, dim=3, lb=[-5,-5,-5], ub=[5,5,5])
best_sol, best_fit, curve = woa.optimize()
print(f"WOA最优解：{best_sol.round(4)}, 适应度：{best_fit:.6f}")